In [ ]:
import tess_cpm
import numpy as np
import matplotlib.pyplot as plt
import lightkurve as lk
from astropy.io import fits
from astropy.wcs import WCS

In [ ]:
# def cpm_periodogram(fits_file, t_row=50, t_col=50):
#     cpm = tess_cpm.CPM(fits_file, remove_bad=True)
#     cpm.set_target(t_row, t_col)
#     cpm.set_exclusion(10)
#     cpm.set_predictor_pixels(256, method='cosine_similarity')
#     cpm.lsq(0.1, rescale=True, polynomials=False)
#     tess_cpm.summary_plot(cpm, 10)
#     aperture_lc, lc_matrix = cpm.get_aperture_lc(box=1, show_pixel_lc=True, show_aperture_lc=True)
#     lc = lk.LightCurve(time=cpm.time, flux=aperture_lc)
#     pg = lc.to_periodogram(oversample_factor=100)
#     fig, axs = plt.subplots(2, 1, figsize=(15, 8))
#     pg.plot(ax=axs[0], c='k')
#     pg.plot(ax=axs[1], c='k', view='period')
#     fig.suptitle("Periodogram", fontsize=20, y=0.95)
#     period = pg.period_at_max_power
#     print(f"Max Power Period: {period}")
#     lc.fold(period.value*4).scatter()
#     plt.title(f"Folded Lightcurve with Period: {period:.4f}", fontsize=20)
#     return cpm

In [ ]:
plt.rcParams["figure.figsize"] = (12, 8)

In [ ]:
fits_file = "dwarfs/tess-s0012-3-3_169.234200_-80.464300_100x100_astrocut.fits"
# fits_file = "dwarfs/tess-s0010-2-1_162.328812_-53.319467_100x100_astrocut.fits"  # provided source
# fits_file = "dwarfs/tess-s0010-3-2_162.328812_-53.319467_100x100_astrocut.fits"  # provided source

In [ ]:
dw = tess_cpm.Source(fits_file)

In [ ]:
dw.plot_cutout()
dw.set_aperture(rowrange=[50, 52], colrange=[50, 52])
# dw.set_aperture(rowrange=[46, 52], colrange=[48, 53])

dw.plot_cutout(rowrange=[45, 55], colrange=[45, 55], show_aperture=True)

In [ ]:
dw.plot_pix_by_pix("normalized_flux")

In [ ]:
dw.add_cpm_model()
dw.set_regs([0.5])
dw.holdout_fit_predict(k=5)
dw.plot_pix_by_pix(data_type="cpm_subtracted_flux")

In [ ]:
detrended_lc = dw.get_aperture_lc(split=True, data_type="cpm_subtracted_flux")
for time, lc in zip(dw.split_times, detrended_lc):
    plt.plot(time, lc, "-")

In [ ]:
cpm_regs = 10.0 ** np.arange(-9, 9)
min_cdpp_reg, cdpps = dw.calc_min_cpm_reg(cpm_regs, k=5)

In [ ]:
print(min_cdpp_reg)
# dw.set_regs([min_cdpp_reg])
dw.set_regs([min_cdpp_reg])

dw.holdout_fit_predict(k=10);

In [ ]:
outliers = dw.get_outliers()

In [ ]:
plt.plot(dw.time, dw.get_aperture_lc(split=False, data_type="normalized_flux"), "-", c="k")
# plt.plot(dw.time, dw.get_aperture_lc(split=False, data_type="cpm_prediction"), "-", c="r")
# plt.plot(dw.time[~outliers], dw.get_aperture_lc(split=False, data_type="cpm_subtracted_flux")[~outliers], "-", c="k")